In [1]:
import os, sys, random, datetime, pickle, joblib, multiprocessing, gc, warnings, inspect, re, itertools

import numpy as np
import pandas as pd

from itertools import compress
from functools import reduce

DATA_PATH = '../../data/'

### Load Data

In [2]:
mastermapping = pd.read_excel(DATA_PATH + 'master_mapping_0918.xlsx')

In [3]:
mastermapping_sics = pd.read_csv(DATA_PATH + 'master_map_sics_filled.csv')

In [4]:
master_esg_calcs = pd.read_csv(DATA_PATH + 'master_esg_calcs.csv')
master_esg_calcs = master_esg_calcs.rename(columns={'RIC_Primary':'RIC_primary'})

C:\Users\jinga\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (789,790,914,915,1047,1048,1057,1061,1065,1067,1069,1072,1076,1103,1176) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
esg_metrics_detail = pd.read_excel(DATA_PATH + 'esg_metrics_detail.xlsx')

C:\Users\jinga\anaconda3\envs\py37\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [6]:
sasb_mapdom = pd.read_excel(DATA_PATH + 'sasb_mapdom.xlsx')

In [7]:
portfolio = pd.read_excel( '../data/franklin_portfolio/franklin_portfolio.xlsx')
portfolio = portfolio[['Name','RIC','RIC_primary','ISIN','our_isin','Weight']]
portfolio = portfolio.dropna(how='all',axis=0)

### Data Info Mapping

In [8]:
to_merge_industry = mastermapping_sics[['RIC_primary','sics_industry_code']].drop_duplicates()

portfolio = portfolio.merge(to_merge_industry,on='RIC_primary',how='left')

portfolio = portfolio.rename(columns={'sics_industry_code':'industry'})

### task 2(b)

#### do we have at least 10 metrics for that company? Master_esg_calcs.csv, only look at ESG metrics, exclude XY Change, /.

#### 只看category_db.isin(['Environment','Social','Governance’], 然后排除名字里带 ‘Y Change’, ‘ / ’ 这样的就可以

In [9]:
#### get E, S, G metrics from category_DB
names_ESG_2b = esg_metrics_detail.loc[esg_metrics_detail['category_db'].isin(['Environmental','Social','Governance']),'calcs_name'].tolist()

#### remove Y Change names
names_ESG_2b = [x for x in names_ESG_2b if 'Y Change' not in x]

#### remove / names
names_ESG_2b = [x for x in names_ESG_2b if '/' not in x]

In [10]:
len(names_ESG_2b)

525

#### select the portfolio's RIC_primary among master_esg_calcs

In [11]:
master_esg_calcs_2b = master_esg_calcs[['RIC_primary','FY'] + names_ESG_2b]

master_esg_calcs_2b = master_esg_calcs_2b.loc[master_esg_calcs_2b['RIC_primary'].isin(portfolio['RIC_primary'])]

master_esg_calcs_2b = master_esg_calcs_2b.reset_index(drop=True)

In [12]:
coverage_2b = master_esg_calcs_2b.groupby(['RIC_primary','FY']).apply(lambda x: x[names_ESG_2b].notna().sum().sum())
coverage_2b = coverage_2b.reset_index().rename(columns={0:'valid_data_points_num'})

In [13]:
coverage_2b['valid_data_points_coverage'] = coverage_2b['valid_data_points_num']/len(names_ESG_2b)

### task 2(c)

#### 要看每个公司对应的industry, 在sasb_mapdom里面有多少个material的trcode, 然后你通过esg_metrics_detail里 trcode和 index_metric=1的可以找到在calc_name. 这些对上的, 是当前这家公司的所有的material的metrics, 然后你再数这些metrics里面, 多少非空

#### 做2c的时候不需要category_db.isin(['Environment','Social','Governance’] 还有  ‘Y Change’ 这个筛选哈, 通过material的mapping 本身就排除了

#### Step 1 ：industry - trcode mapping data

In [14]:
to_merge_trcode = sasb_mapdom[['industry','trcode']].dropna().drop_duplicates()

to_merge_trcode

,industry,trcode
0,RR-PP,TR.AnalyticRenewEnergyUse
1,RR-ST,TR.EcoDesignProducts
2,CG-AM,TR.PercentageGreenProducts
3,CG-BF,TR.PercentageGreenProducts
4,CG-EC,TR.AsianMinoritiesEmployeesPercentage
...,...,...
1138,TR-AP,TR.WasteTotal
1139,TR-RA,TR.EmployeeFatalities
1140,TR-RA,TR.ContractorFatalities
1141,TR-RO,TR.EmployeeFatalities


#### Step 2: 根据trcode，将index_metric merge至上一步结果

In [15]:
esg_metrics_detail_2c = esg_metrics_detail[['trcode','index_metric','calcs_name']].dropna()
esg_metrics_detail_2c['trcode_lower'] = [x.split('(')[0].lower() for x in esg_metrics_detail_2c['trcode']]
esg_metrics_detail_2c

,trcode,index_metric,calcs_name,trcode_lower
0,TR.CO2EmissionTotal,1.0,Total CO2 Emissions,tr.co2emissiontotal
4,TR.AnalyticCO2,1.0,Total CO2 Emissions / Revenue,tr.analyticco2
20,TR.CO2DirectScope1,1.0,"CO2 Emissions, Scope 1 / Revenue",tr.co2directscope1
24,TR.CO2IndirectScope2,1.0,"CO2 Emissions, Scope 2 / Revenue",tr.co2indirectscope2
28,TR.CO2IndirectScope3,1.0,"CO2 Emissions, Scope 3 / Revenue",tr.co2indirectscope3
...,...,...,...,...
905,TR.BlackorAfricanAmericanMinoritiesManagersPer...,1.0,Black or African American - Minorities Manager...,tr.blackorafricanamericanminoritiesmanagersper...
906,TR.HispanicorLatinoMinoritiesManagersPercentage,1.0,Hispanic or Latino - Minorities Managers Perce...,tr.hispanicorlatinominoritiesmanagerspercentage
907,TR.WhiteMinoritiesManagersPercentage,1.0,White - Minorities Managers Percentage,tr.whiteminoritiesmanagerspercentage
908,TR.EnvProducts,1.0,Environmental Products,tr.envproducts


In [16]:
to_merge_trcode['trcode_lower'] = [x.lower() for x in to_merge_trcode['trcode']]
to_merge_trcode

,industry,trcode,trcode_lower
0,RR-PP,TR.AnalyticRenewEnergyUse,tr.analyticrenewenergyuse
1,RR-ST,TR.EcoDesignProducts,tr.ecodesignproducts
2,CG-AM,TR.PercentageGreenProducts,tr.percentagegreenproducts
3,CG-BF,TR.PercentageGreenProducts,tr.percentagegreenproducts
4,CG-EC,TR.AsianMinoritiesEmployeesPercentage,tr.asianminoritiesemployeespercentage
...,...,...,...
1138,TR-AP,TR.WasteTotal,tr.wastetotal
1139,TR-RA,TR.EmployeeFatalities,tr.employeefatalities
1140,TR-RA,TR.ContractorFatalities,tr.contractorfatalities
1141,TR-RO,TR.EmployeeFatalities,tr.employeefatalities


In [17]:
to_merge_trcode = to_merge_trcode.merge(esg_metrics_detail_2c[['trcode_lower','index_metric','calcs_name']].dropna(),on='trcode_lower',how='left')
to_merge_trcode = to_merge_trcode.loc[to_merge_trcode['index_metric']==1]
to_merge_trcode

,industry,trcode,trcode_lower,index_metric,calcs_name
0,RR-PP,TR.AnalyticRenewEnergyUse,tr.analyticrenewenergyuse,1.0,Renewable Energy / Total Energy
1,RR-ST,TR.EcoDesignProducts,tr.ecodesignproducts,1.0,Eco-Design Products
2,CG-AM,TR.PercentageGreenProducts,tr.percentagegreenproducts,1.0,Percentage of Green Products
3,CG-BF,TR.PercentageGreenProducts,tr.percentagegreenproducts,1.0,Percentage of Green Products
4,CG-EC,TR.AsianMinoritiesEmployeesPercentage,tr.asianminoritiesemployeespercentage,1.0,Asian - Minorities Employees Percentage
...,...,...,...,...,...
775,TR-AP,TR.WasteTotal,tr.wastetotal,1.0,Total Waste / Revenue
776,TR-RA,TR.EmployeeFatalities,tr.employeefatalities,1.0,Employee Fatalities / Total Employees
777,TR-RA,TR.ContractorFatalities,tr.contractorfatalities,1.0,Contractor Fatalities / Total Employees
778,TR-RO,TR.EmployeeFatalities,tr.employeefatalities,1.0,Employee Fatalities / Total Employees


In [18]:
to_merge_trcode.index_metric.isna().sum()

0

#### Step 3: 每个公司对应的calcs_name的valid coverage

In [19]:
master_esg_calcs_2c = master_esg_calcs.loc[master_esg_calcs['RIC_primary'].isin(portfolio['RIC_primary'])]
master_esg_calcs_2c = master_esg_calcs_2c.reset_index(drop=True)
master_esg_calcs_2c

,RIC_primary,RIC,Date,FY,Forward PE,Forward PE | 1Y Change,Forward PE | 3Y Change,Forward PE | 5Y Change,Gross Margin,Gross Margin | 1Y Change,...,Debt to Equity,Debt to Equity | 1Y Change,Debt to Equity | 3Y Change,Debt to Equity | 5Y Change,Enterprise Value - Actual,Cash and Equivalents,Date_temp,ret_1yr,ret_3yr,ret_5yr
0,002352.SZ,002352.SZ,2014-12-31 00:00:00+00:00,FY2014,NaN,NaN,NaN,NaN,13.14973,NaN,...,27.251879,NaN,NaN,NaN,NaN,2.264191,201412.0,-0.003148,0.239468,-0.181754
1,002352.SZ,002352.SZ,2015-12-31 00:00:00+00:00,FY2015,NaN,NaN,NaN,NaN,14.35129,1.20156,...,18.633841,-0.316236,NaN,NaN,NaN,3.010670,201512.0,1.059738,1.471525,0.713499
2,002352.SZ,002352.SZ,2016-12-31 00:00:00+00:00,FY2016,72.369338,NaN,NaN,NaN,19.66850,5.31721,...,58.078271,2.116817,NaN,NaN,NaN,1035.868629,201612.0,1.595439,4.329092,5.626102
3,002352.SZ,002352.SZ,2017-12-31 00:00:00+00:00,FY2017,46.730446,-25.638892,NaN,NaN,20.15856,0.49006,...,34.585029,-0.404510,0.269088,NaN,34089.163907,6.586007,201712.0,0.214727,5.493838,6.792100
4,002352.SZ,002352.SZ,2018-12-31 00:00:00+00:00,FY2018,31.620515,-15.109931,NaN,NaN,17.92394,-2.23462,...,46.774030,0.352436,1.510166,NaN,22855.398328,9.004839,201812.0,-0.346711,1.059657,3.228996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,YUMC.N,YUMC.N,2018-12-31 00:00:00+00:00,FY2018,22.420595,-5.595590,NaN,NaN,44.91979,-1.41822,...,0.870171,-0.140707,-0.522401,NaN,12646.000000,1266.000000,201812.0,-0.152907,0.324887,0.324887
905,YUMC.N,YUMC.N,2019-12-31 00:00:00+00:00,FY2019,26.971910,4.551315,5.564531,NaN,43.81267,-1.10712,...,0.909977,0.045746,-0.254132,-0.522767,16492.760000,1046.000000,201912.0,0.448530,0.884606,0.919138
906,YUMC.N,YUMC.N,2020-12-31 00:00:00+00:00,FY2020,34.181330,7.209420,6.165145,NaN,43.05942,-0.75325,...,0.483403,-0.468774,-0.522639,-0.734681,19742.800000,1158.000000,202012.0,0.194685,0.465926,1.292765
907,YUMC.N,YUMC.N,2021-12-31 00:00:00+00:00,FY2021,33.330435,-0.850895,10.909840,11.923056,41.95676,-1.10266,...,0.609410,0.260667,-0.299666,-0.500494,18227.520000,1136.000000,202112.0,-0.119903,0.523041,0.981547


In [20]:
coverage_2c = []
for i in master_esg_calcs_2c['RIC_primary'].unique():
# i = '002352.SZ'
    industry_tmp = portfolio.loc[portfolio['RIC_primary']==i,'industry'].values[0]
    
    columns_tmp = to_merge_trcode.loc[to_merge_trcode['industry']==industry_tmp]['calcs_name'].tolist()
    
    df_tmp = master_esg_calcs_2c.loc[master_esg_calcs_2c['RIC_primary']==i][['RIC_primary','FY']+columns_tmp]
    
    groupby_tmp = df_tmp.groupby(['RIC_primary','FY']).apply(lambda x: pd.DataFrame([x[columns_tmp].notna().sum().sum(),
                                                                        len(columns_tmp),
                                                                        x[columns_tmp].notna().sum().sum()/len(columns_tmp)]).T)
    
    coverage_2c.append(groupby_tmp)

In [21]:
coverage_2c = pd.concat(coverage_2c).reset_index()
coverage_2c = coverage_2c.rename(columns={0:'valid_materials_num',
                                          1:'total_materials_num',
                                          2:'valid_materials_coverage'})
coverage_2c = coverage_2c.drop(columns='level_2')

### 合并

In [22]:
final_output = coverage_2b.merge(coverage_2c,on=['RIC_primary','FY'],how='left')

In [23]:
final_output = final_output.merge(portfolio,on='RIC_primary',how='left')

final_cols = ['Name','RIC_primary','RIC','ISIN','our_isin','industry','Weight',
              'FY','valid_data_points_num','valid_data_points_coverage',
             'valid_materials_num','total_materials_num','valid_materials_coverage']

final_output = final_output[final_cols]

In [24]:
final_output

,Name,RIC_primary,RIC,ISIN,our_isin,industry,Weight,FY,valid_data_points_num,valid_data_points_coverage,valid_materials_num,total_materials_num,valid_materials_coverage
0,"S.F. Holding Co., Ltd.",002352.SZ,002352.SZ,CNE100000L63,CNE100000L63,TR-AF,0.407079,FY2014,1,0.001905,0.0,15.0,0.000000
1,"S.F. Holding Co., Ltd.",002352.SZ,002352.SZ,CNE100000L63,CNE100000L63,TR-AF,0.407079,FY2015,1,0.001905,0.0,15.0,0.000000
2,"S.F. Holding Co., Ltd.",002352.SZ,002352.SZ,CNE100000L63,CNE100000L63,TR-AF,0.407079,FY2016,1,0.001905,0.0,15.0,0.000000
3,"S.F. Holding Co., Ltd.",002352.SZ,002352.SZ,CNE100000L63,CNE100000L63,TR-AF,0.407079,FY2017,1,0.001905,0.0,15.0,0.000000
4,"S.F. Holding Co., Ltd.",002352.SZ,002352.SZ,CNE100000L63,CNE100000L63,TR-AF,0.407079,FY2018,1,0.001905,0.0,15.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,"YUM CHINA HOLDINGS, INC.",YUMC.N,YUMC.N,US98850P1093,US98850P1093,FB-RN,0.400000,FY2018,291,0.554286,4.0,12.0,0.333333
905,"YUM CHINA HOLDINGS, INC.",YUMC.N,YUMC.N,US98850P1093,US98850P1093,FB-RN,0.400000,FY2019,294,0.560000,4.0,12.0,0.333333
906,"YUM CHINA HOLDINGS, INC.",YUMC.N,YUMC.N,US98850P1093,US98850P1093,FB-RN,0.400000,FY2020,305,0.580952,4.0,12.0,0.333333
907,"YUM CHINA HOLDINGS, INC.",YUMC.N,YUMC.N,US98850P1093,US98850P1093,FB-RN,0.400000,FY2021,304,0.579048,4.0,12.0,0.333333


In [26]:
# final_output.to_excel('../data/franklin_portfolio/results/task2_results_0927.xlsx',index=False)